# 3. Customized model

One of the ideas of PiNN is that different atomic neural networks shares similar building blocks.

PiNN's abstraction of layers allows us to construct new network structures from those building blocks.  
It also makes it easier to design novel layers that works along with existing ones.

In this notebook, we will build a customized model by combining PiNN's Pi blocks with Behler's  
element specific neural network. In this way, Pi blocks serves as a "learnable" feature generator 
for BPNN.

In [1]:
import os, warnings
import tensorflow as tf
import pinn.layers as l

from glob import glob
from pinn.models import potential_model
from pinn.networks import pinn_network
from pinn.utils import get_atomic_dress
from pinn.datasets.qm9 import load_QM9_dataset

os.environ['CUDA_VISIBLE_DEVICES'] = ''
index_warning = 'Converting sparse IndexedSlices'
warnings.filterwarnings('ignore', index_warning)

## Defining a custom model function

You can build a customized model simply by constructing a function to map the 
input `tensors` to prediction. Using the defined building blocks will make this
a lot easier. The resulting function can be feed into the potential model, and 
you will be able to train, evaluate and use the model like any defined models. 

In [2]:
def hybrid_network(tensors, atom_types, **kwargs):
    # Use PiNN network function, but get the atomic properties instead of energy
    atom_prop_all = pinn_network(tensors, to_return=1, **kwargs)
    en = 0.0
    n_sample = tf.shape(tensors['atoms'])[0]
    # Use a larger network for energy prediction, and split the energy prediction
    # layers for each element
    for i in atom_types:
        indices = tf.where(tf.equal(tensors['elem'],i))
        atom_prop_i = tf.gather_nd(atom_prop_all, indices)
        sample_id_i = tf.gather_nd(tensors['ind'][1], indices)
        en += l.en_layer(sample_id_i, atom_prop_i, n_sample, [64,64,64],
                         'separate_en_{}'.format(i))
    return en

## Train and evaluation

After that, the model can be readily trained.

In [3]:
filelist = glob('/home/yunqi/datasets/QM9/dsgdb9nsd/*.xyz')
dataset = lambda: load_QM9_dataset(filelist, split_ratio={'train':8, 'test':2})
dress, error = get_atomic_dress(dataset()['train'],[1,6,7,8,9],max_iter=2000)

In [4]:
params = {'model_dir': '/tmp/hybrid_model',
          'network': hybrid_network,
          'netparam': {
              'pre_level': 0,
              'atom_types':[1, 6, 7, 8, 9],
              'atomic_dress': dress},
          'train': {
              'learning_rate': 1e-3,
              'en_scale': 627.5
          }}
config = tf.estimator.RunConfig(log_step_count_steps=500)
model = potential_model(params, config)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/hybrid_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 500, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f17dbb95668>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
# Preprocessing the datasets
pre_fn = lambda tensors: pinn_network(tensors, preprocess=True, **params['netparam'])
train = lambda: dataset()['train'].cache().repeat().shuffle(1000).batch(100).map(pre_fn, 8)
test = lambda: dataset()['test'].cache().repeat().batch(100).map(pre_fn, 8)

# Running specs
train_spec = tf.estimator.TrainSpec(input_fn=train, max_steps=1e4)
eval_spec = tf.estimator.EvalSpec(input_fn=test, steps=100)
tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/hybrid_model/model.ckpt.
INFO:tensorflow:loss = 679.8525, step = 1
INFO:tensorflow:global_step/sec: 8.50666
INFO:tensorflow:loss = 133.58022, step = 501 (58.783 sec)
INFO:tensorflow:global_step/sec: 8.5613
INFO:tensorflow:loss = 186.6181, step = 1001 (58.401 sec)
INFO:tensorflow:global_step/sec: 15.0342
INFO:tensorflow:loss = 173.83643, step = 1501 (33.256 sec)
INFO

({'ENG_MAE': 3.7595446,
  'ENG_RMSE': 5.360081,
  'loss': 28.730474,
  'global_step': 10000},
 [])

## Conclusions

Our hybrid model beats the original PiNN at the cost of slower training for this test, 
although both model have not reached their full potential. Feel free to look into the 
definitions of `networks` & their `layers` and improve them with your custom models!

**Note** You can reuse a model given the same network function, however, there's 
currently no way to recover a model if you lost you original function definition.